# Plot XRF maps from flyscans without fitting

This notebook will plot XRF  maps from flyscans. It requires data from the `.h5` masterfile, the `_falconx.h5` file.

Plots are based on selections of regions of interest

The user should provide a list of scan IDs, which will then be read and stitched together.

Two ROIs should be defined in the various cells, one for the interval to use for the XRF maps (in keV).

NB: This notebooks tries to stitch maps in the data type (*float32*). This works for small dataset with low count rate.
For larger datasets, it might be impossible to hold them in memory. try changing the `map_type` paramter of the `stitch_scan` function to `np.uint16`


### For XRF

Note that this notebook makes RoIs based on the XRF spectrum, it does not fit the XRF data.
For XRF fitting use the notebook "mapping_XRF_fitting"

### Content
[**Import and reshape data**](#import)  
[**Plot average fluorescence spectrum**](#avg_spec)  
[**Plot XRF maps**](#xrf_map)  
[**Plot average diffraction pattern**](#avg_xrd)  
[**Plot XRD maps**](#xrd_map)  

In [ ]:
%matplotlib widget
import os
import sys
import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from IPython.utils import io

#To import DanMAX from the folder above:
sys.path.append('../')
import DanMAX as DM
style = DM.darkMode(style_dic={'figure.figsize':'large'})

In [ ]:
# Define list of scan IDs:
scans = [XXX]
proposal,visit = DM.getCurrentProposal()


#Select emmission lines to plot:
xrf_rois = {'Ca'    : ( 3.4 ,  4.2),
            'Zn'    : ( 8.45,  9.9),
            'Zn_Kb' : ( 9.4 ,  9.7),
            'Sr'    : (13.85, 14.5)}

# 'Ar'    : ( 2.8 ,  3.0 )
# 'Ca'    : ( 3.4 ,  4.2 )
# 'Ca_Ka' : ( 3.5 ,  3.8 )
# 'Ca_Kb' : ( 3.84,  4.04)
# 'Zn'    : ( 8.45,  9.9 )
# 'Zn_Ka' : ( 8.45,  8.70)
# 'Zn_Kb' : ( 9.4 ,  9.7 )
# 'Br'    : (11.6 , 12.1 )
# 'Sr'    : (13.85, 14.5 )

#### import and reshape data  <a id='import'></a>

In [ ]:
# import falcon x data

snitch = DM.mapping.stitchScans(scans,proposal=proposal,visit=visit,XRD=False)
xx = snitch['x_map']
yy = snitch['y_map']
xrf_map = snitch['xrf_map']
energy = snitch['energy']
Emax = snitch['Emax']
I0_map = snitch['I0_map']

del snitch
#Apply I0 correction
xrf_map = (xrf_map.transpose(2,0,1)/ I0_map).transpose(1,2,0)
                                            
#use only the correct part of the energyvector
energy=energy[energy<Emax*1.1]
# calculate average spectrum
xrf_avg = np.nanmean(xrf_map,axis=(0,1))
print(f'Stitched image dimensions: {xrf_map.shape}')

#### plot the average spectrum  <a id='avg_spec'></a>

In [ ]:

plt.figure()
plt.title('Average spectrum')
plt.plot(energy,xrf_avg)
plt.xlabel('Energy (keV)')
plt.ylabel('Intensity')
plt.xlim(0,Emax*1.1)
# plot emmission lines
print('Selected emmission lines:')
for key in xrf_rois:
    xrf_roi = (energy > xrf_rois[key][0]) & (energy < xrf_rois[key][1])
    plt.axvline(xrf_rois[key][0],c='k',ls='--',lw=1)
    plt.axvline(xrf_rois[key][1],c='k',ls='--',lw=1)
    plt.plot(energy[xrf_roi],xrf_avg[xrf_roi], '.',ms=2,label=f'{key:<8s}: {lines[key][0]:4.1f} - {lines[key][1]:4.1f} keV')
plt.legend()

#### plot XRF heatmaps  <a id='xrf_map'></a>

In [ ]:
# Set the number of columns for the figure
cols = 4 

rows = int(len(xrf_rois)/cols) + (len(xrf_rois)%cols!=0)
# initialize subplots with shared x- and y-axes
fig,axs = plt.subplots(rows,cols,sharex=True,sharey=True)
fig.tight_layout()
axs = axs.flatten() # flatten the axes list to make it easier to index

for i,key in enumerate(xrf_rois):
    lower, upper = xrf_rois[key]
    lower, upper = np.digitize(lower, energy), np.digitize(upper, energy)

    # reshape the fluorescence signal from the falcon x
    #S  = np.mean(falconx[:,lower:upper],axis=1).reshape(map_shape)
    xrf = np.mean(xrf_map[:,:,lower:upper],axis=2)
    # plot the map as a pseudo colormesh
    ax = axs[i]
    ax.set_title(key)
    pcm = ax.pcolormesh(xx,
                        yy,
                        xrf,
                        norm='log',
                        shading='nearest')
    fig.colorbar(pcm,ax=ax,aspect=40,pad=0.05)
    ax.set_xlabel('x mm')
    ax.set_ylabel('y mm')
    # set the aspect ratio to equal to give square pixels
    ax.set_aspect('equal')

# delete surplus plots
for i in range(1,cols*rows-len(lines)+1):
    fig.delaxes(axs[-i])